Using Time Series LSTM for python I am going to predict the potential future amount of parts within the materials in the Mt. Holly Plant

In [1]:
import gspread
import numpy as np
import pyspark.pandas as ps
import numba
from numba import jit
import os
import json
from datetime import *

First task is to import the data into a Spark Dataframe & clean the dataframe

In [10]:
#setup spark env & import spark variables
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/27 01:40:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
#call to the dataset within Google Sheets
sa = gspread.service_account(filename="/com.docker.devenvironments.code/kuthesisdataclean-558333f9362c.json")

In [35]:
@jit
def get_data(sa):
    sheet = sa.open("POU_Unclean_Data")
    work_sheet = sheet.worksheet("POU")
    rows = work_sheet.get_all_values()
    columns = ['Item_no', 'Item_description', 'Quantity', 'Status', 'Serial Number','Reason Code', 'Shortage_notes', 'Team_found', 'Team_Resp', 'Date_Found', 'Time_Found_Pacific','short_found_emp_id', 'date_chassis_start', 'date_veh_offln', 'date_mfg_rise', 'prim_shop_addr', 'scd_shop_addr_1', 'scd_shop_addr_2', 'scd_shop_addr_3', 'scd_shop_addr_4']
    df = spark.createDataFrame(rows, columns)
    df = df.where(df.Item_no!='Item_no')
    return df

In [36]:
df = get_data(sa)


/tmp/ipykernel_16043/786872232.py:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "get_data" failed type inference due to: Untyped global name 'spark': Cannot determine Numba type of <class 'pyspark.sql.session.SparkSession'>

File "tmp/ipykernel_16043/786872232.py", line 7:
<source missing, REPL/exec in use?>

  @jit
/home/vscode/micromamba/envs/py39/lib/python3.10/site-packages/numba/core/object_mode_passes.py:151: NumbaWarning: Function "get_data" was compiled in object mode without forceobj=True.

File "tmp/ipykernel_16043/786872232.py", line 1:
<source missing, REPL/exec in use?>

  warnings.warn(errors.NumbaWarning(warn_msg,
/home/vscode/micromamba/envs/py39/lib/python3.10/site-packages/numba/core/object_mode_passes.py:161: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected, this is deprecated behaviour.

For more information visit https://numba.readthe

In [39]:
df.printSchema()
#df.show()


root
 |-- Item_no: string (nullable = true)
 |-- Item_description: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Serial Number: string (nullable = true)
 |-- Reason Code: string (nullable = true)
 |-- Shortage_notes: string (nullable = true)
 |-- Team_found: string (nullable = true)
 |-- Team_Resp: string (nullable = true)
 |-- Date_Found: string (nullable = true)
 |-- Time_Found_Pacific: string (nullable = true)
 |-- short_found_emp_id: string (nullable = true)
 |-- date_chassis_start: string (nullable = true)
 |-- date_veh_offln: string (nullable = true)
 |-- date_mfg_rise: string (nullable = true)
 |-- prim_shop_addr: string (nullable = true)
 |-- scd_shop_addr_1: string (nullable = true)
 |-- scd_shop_addr_2: string (nullable = true)
 |-- scd_shop_addr_3: string (nullable = true)
 |-- scd_shop_addr_4: string (nullable = true)

